In [1]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, callback_context
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bootstrap_components as dbc
import calendar
import datetime
import requests
from io import StringIO

In [2]:
# Fetch Ferry Data

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
 
# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "toronto-island-ferry-ticket-counts"}
package = requests.get(url, params = params).json()
 
# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):
 
       # for datastore_active resources:
       if resource["datastore_active"]:
 
           # To get all records in CSV format:
           url = base_url + "/datastore/dump/" + resource["id"]
           resource_dump_data = requests.get(url).text
 
           # To selectively pull records and attribute-level metadata:
           url = base_url + "/api/3/action/datastore_search"
           p = { "id": resource["id"] }
           resource_search_data = requests.get(url, params = p).json()["result"]

           # This API call has many parameters. They're documented here:
           # https://docs.ckan.org/en/latest/maintaining/datastore.html
 
       # To get metadata for non datastore_active resources:
       if not resource["datastore_active"]:
           url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
           resource_metadata = requests.get(url).json()
           # From here, you can use the "url" attribute to download this file



In [3]:
# Load & process data
def load_data():
    df = pd.read_csv(StringIO(resource_dump_data))
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['Year'] = df['Timestamp'].dt.year
    df['Month_Num'] = df['Timestamp'].dt.month
    df['Month'] = df['Month_Num'].apply(lambda x: calendar.month_name[x])
    df['Day'] = df['Timestamp'].dt.date
    df['Hour'] = df['Timestamp'].dt.hour
    df['Minute'] = df['Timestamp'].dt.strftime('%H:%M')
    df['rounded_time'] = df['Timestamp'].dt.floor('h')

    weather_df = pd.read_csv('weather_data.csv')
    weather_df['time'] = pd.to_datetime(weather_df['time'])

    merged = pd.merge(df, weather_df, left_on='rounded_time', right_on='time', how='left')
    return df, weather_df, merged

In [4]:
# Initial load
df, weather_df, merged = load_data()

In [5]:
# App setup
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

In [6]:
# Drill state
drill_state = []
drill_levels = ['Year', 'Month', 'Day', 'Hour', 'Minute']
drill_titles = [
    'Total Redemptions by Year',
    'Monthly Redemptions in {}',
    'Daily Redemptions in {} {}',
    'Hourly Redemptions on {}',
    'Minute-Level Redemptions on {}'
]

# Layout
app.layout = dbc.Container([
    dbc.NavbarSimple(brand="Toronto Ferry Dashboard", color="primary", dark=True),
    dcc.Interval(id='midnight-check', interval=60*1000, n_intervals=0),  # Every 1 minute
    dcc.Store(id='dummy-store'),  # Used to trigger callback without affecting layout

    dbc.Row([
        dbc.Col([
            html.H4("Redemption Counts"),
            html.Button("Back", id="back-button", n_clicks=0),
            dcc.Graph(id="bar-graph")
        ], width=8)
    ])
], fluid=True)

# Graph Update Callback (Drilldown)
@app.callback(
    Output("bar-graph", "figure"),
    Input("bar-graph", "clickData"),
    Input("back-button", "n_clicks"),
    State("bar-graph", "figure")
)
def update_bar_graph(clickData, n_clicks, current_fig):
    global drill_state, df

    ctx = callback_context
    trigger = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None

    if trigger == "back-button" and drill_state:
        drill_state.pop()
    elif trigger == "bar-graph" and clickData is not None and len(drill_state) < 4:
        clicked_value = str(clickData['points'][0]['x'])
        drill_state.append(clicked_value)

    level = len(drill_state)

    if level == 0:
        df_grouped = df.groupby("Year")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped, x="Year", y="Redemption Count", title=drill_titles[0])
    elif level == 1:
        df_filtered = df[df['Year'] == int(drill_state[0])]
        df_grouped = df_filtered.groupby(['Month', 'Month_Num'])['Redemption Count'].sum().reset_index()
        df_grouped = df_grouped.sort_values(by='Month_Num')
        fig = px.bar(df_grouped, x="Month", y="Redemption Count", title=drill_titles[1].format(drill_state[0]))
    elif level == 2:
        df_filtered = df[(df['Year'] == int(drill_state[0])) & (df['Month'] == drill_state[1])]
        df_grouped = df_filtered.groupby("Day")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped.sort_values(by="Day"), x="Day", y="Redemption Count", title=drill_titles[2].format(drill_state[1], drill_state[0]))
    elif level == 3:
        df_filtered = df[df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()]
        df_grouped = df_filtered.groupby("Hour")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped, x="Hour", y="Redemption Count", title=drill_titles[3].format(drill_state[2]))
    elif level == 4:
        df_filtered = df[
            (df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()) &
            (df['Hour'] == int(drill_state[3]))
        ]
        df_grouped = df_filtered.groupby("Timestamp")['Redemption Count'].sum().reset_index()
        df_grouped["label"] = df_grouped["Timestamp"].dt.strftime('%H:%M')
        fig = px.bar(df_grouped, x="label", y="Redemption Count", title=drill_titles[4].format(drill_state[2]))
    else:
        return dash.no_update

    fig.update_xaxes(type='category')
    return fig

# Midnight Auto-Refresh Callback
@app.callback(
    Output("dummy-store", "data"),
    Input("midnight-check", "n_intervals"),
    prevent_initial_call=True
)
def refresh_data(n):
    now = datetime.datetime.now()
    if now.hour == 0 and now.minute < 10:
        global df, weather_df, merged
        df, weather_df, merged = load_data()
        print("✅ Data refreshed at midnight")
        return {"refreshed": True}
    return dash.no_update

In [ ]:
# server = app.server

# Run app locally
if __name__ == '__main__':
    app.run(debug=True, port=8050)

: 